In [1]:
import torch
import numpy as np
from tqdm import tqdm
from scipy.stats.distributions import chi2
from scipy.stats import pearsonr
import pickle

import matplotlib.pyplot as plt
from scipy.stats import linregress
import seaborn as sns
import random
import matplotlib.animation as animation
import sys, os

path = os.path.dirname(os.getcwd())
sys.path.append(path)

from utils.plots import *
from utils.funcs import *
from utils.visualize_network import *
from utils.visualize_hidden_network import *

from boltzmann_machines.RTRBM_ import RTRBM as RTRBM_
from boltzmann_machines.RTRBM import RTRBM
from boltzmann_machines.RTRBM_no_bias import RTRBM_nobias
from boltzmann_machines.RBM import RBM

from data.load_data import *
from data.mock_data import *
from data.reshape_data import *

# spikes, behavior, coordinates, df, stimulus = loaddata()

"""
to dump:

path = os.path.dirname(os.getcwd())
pickle.dump(rtrbm, open(path + '\\results\\RTRBMs\\RTRBMs1', 'wb'))

to load:


check1 = pickle.load(open(path + '\\results\\RTRBMs\\RTRBMs1', 'rb'))
"""

"\nto dump:\n\npath = os.path.dirname(os.getcwd())\npickle.dump(rtrbm, open(path + '\\results\\RTRBMs\\RTRBMs1', 'wb'))\n\nto load:\n\n\ncheck1 = pickle.load(open(path + '\\results\\RTRBMs\\RTRBMs1', 'rb'))\n"

How many batches are nessesary to accurately converge and to determine the underlying structure

In [3]:
######## Simulation parameters ########
neurons_per_pop = 20
n_pop = 6
n_batches = 1000
T = 50
mean_fr = .1

######## Defining coordinate system ########
rads = torch.linspace(0, 2*torch.pi, n_pop+1)
mean_locations_pop = torch.zeros(n_pop, 2)
coordinates = torch.zeros(neurons_per_pop*n_pop, 2)
for i in range(n_pop):
    mean_locations_pop[i, :] = torch.tensor([torch.cos(rads[i]), torch.sin(rads[i])])
    coordinates[neurons_per_pop * i:neurons_per_pop * (i + 1), :] = 0.15 * torch.randn(neurons_per_pop, 2) + mean_locations_pop[i]

######## Start creating data ########
Data = torch.zeros(neurons_per_pop*n_pop, T, n_batches)
for batch in range(n_batches):

    ######## Creating random input currents and mother trains ########
    t = np.linspace(0, 10*np.pi, T)
    fr = np.zeros((n_pop, T))
    mother = np.zeros((n_pop, T))
    for pop in range(n_pop):
        u = np.random.rand()
        phase = np.random.randn()
        amp = .1*np.random.rand()
        shift = .3*np.random.rand()
        fr[pop, :] = amp*np.sin(phase*(t + 2*np.pi*u)) + shift
        while np.min(fr[pop, :]) < 0:
            u = np.random.rand()
            phase = np.random.randn()
            amp = .1*np.random.rand()
            shift = .3*np.random.rand()
            fr[pop, :] = amp*np.sin(phase*(t + 2*np.pi*u)) + shift
        mother[pop, :] = np.random.poisson(fr[pop, :])


    # empty data array
    spikes = np.zeros((neurons_per_pop*n_pop, T))

    # Excitatory and inhibitory connections
    inh = [[0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]]
    exc = [[3, 4, 5], [3, 4, 5], [3, 4, 5], [3, 4, 5], [3, 4, 5], [3, 4, 5]]
    for pop in range(n_pop):
        delete_spikes = np.roll(np.sum(fr[inh[pop], :], 0), 1) * np.ones((neurons_per_pop, T)) >= np.random.uniform(0, 1, size=(neurons_per_pop, T))
        noise = np.random.poisson(np.roll(np.sum(fr[exc[pop], :], 0), 1), (neurons_per_pop, T))
        temp = np.tile(mother[pop, :], (neurons_per_pop, 1)) - delete_spikes + noise
        spikes[pop*neurons_per_pop:(pop+1)*neurons_per_pop, :] = temp[np.argsort(np.mean(temp, 1)), :]
    spikes[spikes < 0] = 0
    spikes[spikes > 1] = 1

    Data[:, :, batch] = torch.tensor(spikes)

In [3]:
N_V = 20
T = 30
N_H = 3

max_epoch = 1000
n_rtrbms = 2
rtrbm_list_batchsize = []
rtrbm_original_list_batchsize = []

for batchsize in tqdm([10, 50, 100, 150, 200, 250, 350, 500][::-1]):
    print(batchsize)
    rtrbm_list = []
    rtrbm_original_list = []
    
    for _ in range(n_rtrbms):
        data = Data[:, :, np.random.permutation(n_batches)[:batchsize]]
        rtrbm_ = RTRBM(data, N_H=N_H, device='cpu')
        rtrbm_.learn(max_epochs=100, batchsize=1, CDk=10, lr=1e-4, mom=0.9, wc=0.0002)
        
        rtrbm_list.append(rtrbm_)
        rtrbm_original_list.append(rtrbm)
        
    rtrbm_list_batchsize.append(rtrbm_list) 
    rtrbm_original_list_batchsize.append(rtrbm_original_list)   
        
    
# Calculate Pearson correlation of the trained weights



  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

500


  0%|                                                                                            | 0/8 [08:42<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
n_batches = len(rtrbm_original_list_batchsize)
n_rtrbms = len(rtrbm_original_list_batchsize[0])
r_VH = torch.zeros([n_batches, n_rtrbms])
r_HH = torch.zeros([n_batches, n_rtrbms])

for batch in tqdm(range(n_batches)):
    for m in range(n_rtrbms):
        W_original = rtrbm_original_list_batchsize[batch][m].VH.detach().clone()
        W_train = rtrbm_list_batchsize[batch][m].VH.detach().clone()
        W_acc_original = rtrbm_original_list_batchsize[batch][m].HH.detach().clone()
        W_acc_train = rtrbm_original_list_batchsize[batch][m].HH.detach().clone()

        corr = np.zeros((N_H, N_H))
        shuffle_idx = np.zeros((N_H))
        for i in range(N_H):
            for j in range(N_H):
                corr[i, j] = np.correlate(W_original[i,:], W_train[j,:])
            shuffle_idx[i] = np.argmax(corr[i,:])

        W_train = W_train[shuffle_idx, :]    
        W_acc_train = W_acc_train[shuffle_idx, :]
        W_acc_train = W_acc_train[:, shuffle_idx]

        _, _, r_VH[batch, m], _, _ = linregress(W_original.ravel(), W_train.ravel())
        _, _, r_HH[batch, m], _, _ = linregress(W_acc_original.ravel(), W_acc_train.ravel())
        
print(r_VH, r_HH)